In [2]:
import os
# Leer variables de entorno
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DB = os.getenv("PG_DB")
PG_USER = os.getenv("PG_USER")
PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_SCHEMA_RAW = os.getenv("PG_SCHEMA_RAW")
RUN_ID = os.getenv("RUN_ID")

In [3]:
#1. ver fechas minimas y maximas
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB}")
df = pd.read_sql("SELECT * FROM analytics.daily_features", engine)

df.groupby("ticker").agg(["min","max"])[["date"]]


date           
              min        max
ticker                      
AAPL   2019-01-02 2025-10-31
MSFT   2019-01-02 2025-10-31
TSLA   2019-01-02 2025-10-31

In [6]:
#2. Cobertura de fechas raw vs analytics
raw = pd.read_sql("SELECT * FROM raw.prices_daily", engine)
ana = pd.read_sql("SELECT * FROM analytics.daily_features", engine)

raw.groupby("ticker")["date"].agg(["min","max"])



,min,max
ticker,,
AAPL,2019-01-02,2025-10-31
MSFT,2019-01-02,2025-10-31
TSLA,2019-01-02,2025-10-31


In [7]:
ana.groupby("ticker")["date"].agg(["min","max"])

,min,max
ticker,,
AAPL,2019-01-02,2025-10-31
MSFT,2019-01-02,2025-10-31
TSLA,2019-01-02,2025-10-31


In [10]:
# Dias bursátiles sin datos
import pandas as pd

raw["date"] = pd.to_datetime(raw["date"])
ana["date"] = pd.to_datetime(ana["date"])

missing = pd.merge(
    raw[["date","ticker"]],
    ana[["date","ticker"]],
    on=["date","ticker"],
    how="left",
    indicator=True
)


missing_days = missing[missing["_merge"] == "left_only"]
missing_days.head()


,date,ticker,_merge


In [11]:
missing_days.shape

(0, 3)